
# TensorFlow Object Detection: Обучение модели

https://github.com/tensorflow/models/tree/master/research/object_detection

### Монтирование Google Drive
Для данного демо нам понадобятся несколько ноутбуков, которые должны работать с одними и теме же данными. Поэтому, чтобы все ноутбуки имели доступ к нужным данным, нам будет необходимо подключить диск Google Drive и сохранять все данные на нём (включая данные, скачанные из интернета).

Для монтирования диска нужно выполнить данный блок, перейти по ссылке, получить код, скопировать его в поле ниже (в этом блоке) и нажать Enter

После монтирования диск будет находиться здесь: `/content/drive/My Drive`

In [1]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [2]:
pip install tf_slim

     |████████████████████████████████| 352 kB 8.9 MB/s 


In [3]:
pip install lvis

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Рабочая директория
Все данные будем хранить в директории `/content/drive/My Drive/tf_od_demo` (TensorFlow Object Detection Demo)

Директория должна быть уже создана (в предыдущем ноутбуке)

In [5]:
%cd "/content/drive/My Drive/tf_od_demo"

/content/drive/My Drive/tf_od_demo


### Загрузка предобученной модели
Во время обучения мы будем делать fine-tuning предобученной модели. Для этого необходимо загрузить веса соответствующей модели `ssd_mobilenet_v1_coco`. Из названия следует, что детекционная архитектура -- `SSD`, базовая CNN модель -- `MobileNet_v1`, предобучена на датасете COCO.

Этот шаг нужно сделать один раз (не повторять, если модель уже была скачана ранее).

In [6]:
if True:
    !wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz
    !tar -xzf ssd_mobilenet_v1_coco_2018_01_28.tar.gz

--2021-07-27 16:39:59--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.142.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76541073 (73M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_coco_2018_01_28.tar.gz’

ssd_mobilenet_v1_co 100%[===================>]  73.00M  81.7MB/s    in 0.9s    

2021-07-27 16:40:00 (81.7 MB/s) - ‘ssd_mobilenet_v1_coco_2018_01_28.tar.gz’ saved [76541073/76541073]



### Подготовка данных для обучения
Ранее мы подготовили файл `train_data.record`, в котором содержится обучающий датасет.
Кроме него для обучения необходимо подготовить еще два файла (они также находятся в папке `my_data`)

* `label_map.pbtxt` (в нашем случае `cube_label_map.pbtxt`) - файл с описанием классов. В нашем случае у нас всего один класс.

* `pipeline.config` - файл, содержищий различне гиперпарамтеры и настройки обучения. Важные параметры: `num_classes`, `fine_tune_checkpoint`, `label_map_path`, `input_path`. В данном примере и для обучения и для теста используется один и тот же датасет (см. блоки параметров `train_input_reader`, `eval_input_reader`)

### Запуск обучения
После того, как мы подготовили все данные, запустить обучение очень просто: надо просто запустить скрипт `model_main.py` и передать ему соответствующие параметры.

*   Путь к файлу `pipeline.config`
*   Директорию, для записи выхода обучения (`model_dir`)
*   Количество итераций обучения (`num_train_steps`)



In [7]:
%%time

if False:
    # Delete output directory
    !rm -rf my_data2/output

!export PYTHONPATH=$PYTHONPATH:models/research:models/research/slim ; python models/research/object_detection/model_main.py \
    --pipeline_config_path=my_data2/pipeline.config \
    --model_dir=my_data2/output \
    --num_train_steps=40000 \
    --alsologtostderr

W0727 16:41:11.189769 140511013476224 model_lib.py:817] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 40000
I0727 16:41:11.190023 140511013476224 config_util.py:552] Maybe overwriting train_steps: 40000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0727 16:41:11.190129 140511013476224 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0727 16:41:11.190211 140511013476224 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0727 16:41:11.190317 140511013476224 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0727 16:41:11.190429 140511013476224 model_lib.py:833] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to

### Заморозка модели
Для того, чтобы использовать обученную модель в режиме инференса, её нужно "заморозить". Другими словами - подготовить к инференсу. Для этого надо запустить скрипт `export_inference_graph.py` и передать соответствующие парамтеры.

* Путь к файлу `pipeline.config`
* Путь к весам обученной модели (`trained_checkpoint_prefix`)
* Путь к директории для сохранения замороженной модели (`output_directory`)

In [8]:
!export PYTHONPATH=$PYTHONPATH:models/research:models/research/slim ; python models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=my_data2/pipeline.config \
    --trained_checkpoint_prefix=my_data2/output/model.ckpt-40000 \
    --output_directory=my_data2/output/frozen/

Instructions for updating:
Please use `layer.__call__` method instead.
W0727 17:52:30.128393 139753383442304 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0727 17:52:31.530020 139753383442304 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0727 17:52:31.575932 139753383442304 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0727 17:52:31.615930 139753383442304 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo